In [ ]:
%load_ext autoreload
%autoreload 2

from math import pi
from glob import glob

import numpy as np
import matplotlib.pyplot as plt

import uproot
import pandas

In [ ]:
from matplotlib.colors import LogNorm

In [ ]:
plt.rcParams.update({'font.size': 18})

In [ ]:
import scipy.signal

In [ ]:
def GAMMA(MASS,KE):
    return (MASS+KE) / MASS
def BETA(gamma):
    return np.sqrt((gamma**2-1)/gamma**2)

In [ ]:
# Constants

KAON_MASS = 493.677 # MeV
PION_MASS = 139.57  # MeV
MUON_MASS = 105.658 # MeV
ELEC_MASS = 0.511   # MeV

In [ ]:
# load neutrino flux from MicroBooNE

In [ ]:
flux_data = np.loadtxt('MicroBooNE_Flux_Pions_Kaons.csv',skiprows=2,delimiter=',')

In [ ]:
# flux in units of neutrinos / POT / GeV / cm2
kaon_flux = []
pion_flux = []
for flux_row in flux_data:
    kaon_f = flux_row[1]
    pion_f = flux_row[3]
    kaon_flux.append(kaon_f)
    pion_flux.append(pion_f)

In [ ]:
fig = plt.figure(figsize=(6,6))
xvals = np.linspace(0.025,4.925,99)
#print(xvals)
#print (len(xvals))
plt.plot(xvals,pion_flux,'bo')
plt.plot(xvals,kaon_flux,'ro')
plt.yscale('log')
plt.xlabel('Energy [GeV]')
plt.ylabel('Flux [neutrino/POT/GeV/cm2]')
plt.grid(which='both')
plt.show()

In [ ]:
# HNL arrival time
# based on distance L
L = 400 # meters
c = 3e8 # m/sec
# return in nanoseconds

# absolute travel time
def HNL_TIME(MASS,KE,DISTANCE=L):
    #KE = ENERGY - MASS
    E = MASS + KE
    #gamma = GAMMA(MASS,KE)
    #beta = BETA(gamma)
    time = DISTANCE * E / (c * np.sqrt(E**2 - MASS**2))
    return 1e9 * (time )

# relative travel time w.r.t. neutrino
def HNL_DELTATIME(MASS,KE,DISTANCE=L):
    #KE = ENERGY - MASS
    E = MASS + KE
    #gamma = GAMMA(MASS,KE)
    #beta = BETA(gamma)
    time = DISTANCE * E / (c * np.sqrt(E**2 - MASS**2))
    return 1e9 * (time - (DISTANCE/c))
#'''

HNL_DELTATIME_v = np.vectorize(HNL_DELTATIME)
HNL_TIME_v = np.vectorize(HNL_TIME)

In [ ]:
print (HNL_TIME(100,100))
print (HNL_DELTATIME(10,500,DISTANCE=500))

In [ ]:
print (HNL_TIME(100,100,DISTANCE=500))

In [ ]:
# sample bucket in beam
# return time of bucket mean based on frequency
FREQ = 18.936 # ns
def BUCKET(N):
    return np.random.randint(0,22,N) * FREQ

In [ ]:
# sample RWM
SIGMA = 2.5
def RWM(sigma,N):
    return np.random.normal(0,SIGMA,N)

In [ ]:
# sample energy of new particle from neutrino flux
KEMIN = 0. # MeV
KEMAX = 500. # MeV
def ENERGY(N):
    return np.random.uniform(KEMIN,KEMAX,N)

MAXFLUXPION = np.max(pion_flux)
MAXFLUXKAON = np.max(kaon_flux)
BINWIDTH = 0.05 # GeV

# returns kinetic Energy [MeV]
def ENERGY_PION(MASS,N):
    simulated_v = []
    while len(simulated_v) < N:
        # simulate an energy from 0 to 3 GeV
        energy = np.random.uniform(MASS/1e3,2.95)
        # find the bin it falls in [50 MeV bins]
        energybin = int(energy/BINWIDTH)
        # probability of seeing an event in this energy bin
        p = np.random.uniform(0,MAXFLUXPION)
        #print (' p is ',p)
        # is this value smaller than flux prediction at this energy? 
        # if so, continue to simulate event
        if (p < pion_flux[energybin]):
            simulated_v.append(energy*1e3-MASS)
    return np.array(simulated_v)

# returns kinetic Energy [MeV]
def ENERGY_KAON(MASS,N):
    simulated_v = []
    while len(simulated_v) < N:
        # simulate an energy from 0 to 5 GeV
        energy = np.random.uniform(MASS/1e3,4.95)
        # find the bin it falls in [50 MeV bins]
        energybin = int(energy/BINWIDTH)
        # probability of seeing an event in this energy bin
        p = np.random.uniform(0,MAXFLUXKAON)
        #print (' p is ',p)
        # is this value smaller than flux prediction at this energy? 
        # if so, continue to simulate event
        if (p < kaon_flux[energybin]):
            simulated_v.append(energy*1e3-MASS)
    return np.array(simulated_v)

In [ ]:
# input are the masses of the meson, HNL, and lepton respectively
def RHO(MESON,HNL,LEPTON):
    x = (LEPTON/MESON)**2
    y = (HNL/MESON)**2
    return ((x+y-(x-y)**2)) * np.sqrt(1+x**2+y**2-2*(x+y+x*y)) / (x*(1-x)**2)

In [ ]:
RHO_V = np.vectorize(RHO)

In [ ]:
fig = plt.figure(figsize=(6,6))
xvals = np.linspace(1,500,10000) # MeV

yvals = RHO_V(PION_MASS,xvals,MUON_MASS)
plt.plot(xvals,yvals,label=r'$\pi^{+} \rightarrow N + \mu^{+}$')
yvals = RHO_V(PION_MASS,xvals,ELEC_MASS)
plt.plot(xvals,yvals,label=r'$\pi^{+} \rightarrow N + e^{+}$')

yvals = RHO_V(KAON_MASS,xvals,MUON_MASS)
plt.plot(xvals,yvals,label=r'$K^{+} \rightarrow N + \mu^{+}$')
yvals = RHO_V(KAON_MASS,xvals,ELEC_MASS)
plt.plot(xvals,yvals,label=r'$K^{+} \rightarrow N + e^{+}$')

plt.axvline(PION_MASS,color='k',linestyle='--',label=r'$\pi^{+}$ mass')
plt.axvline(KAON_MASS,color='b',linestyle='--',label=r'$K^{+}$ mass')
plt.yscale('log')
plt.xlabel('HNL Mass [MeV]')
plt.ylabel('Branching Ratio [A.U.]')
plt.legend(loc='best',fontsize=12)
plt.ylim([1e-1,2e5])
plt.show()
print (RHO(135,100,0.5)) # pion, HNL, electron

In [ ]:
pion_v = ENERGY_PION(500,100000)
kaon_v = ENERGY_KAON(500,100000)

fig = plt.figure(figsize=(6,6))
xvals = np.linspace(25,4925,99)
#print(xvals)
#print (len(xvals))
plt.hist(pion_v,bins=xvals,histtype='step',lw=2,color='b')
plt.hist(kaon_v,bins=xvals,histtype='step',lw=2,color='r')
plt.yscale('log')
plt.xlabel('Kinetic Energy [MeV]')
plt.ylabel('Flux [neutrino/POT/GeV/cm2]')
plt.grid(which='both')
plt.show()

In [ ]:
# simulate neutrinos
N_nu = 3000000
tnu_v = BUCKET(N_nu) + RWM(SIGMA,N_nu)

# HNL Properties: Mass, $U_{{\alpha}N}$

In [ ]:
MASS_V = np.array([50,150]) # MeV
UaN = 0.5

In [ ]:
# Calculate Branching ratio w.r.t. Neutrino Flux
BR_U = 1. #((UaN**2)/(1-UaN**2))
BR_HNL_V = []

for MASS in MASS_V:
    BR_KIN = 1. #RHO(KAON_MASS,MASS,MUON_MASS)

    BR_HNL_V.append( BR_KIN * BR_U )

In [ ]:
# simualte HNLs

N_HNL_V = []
HNL_KE_v_V = []
thnl_v_V = []

for i,MASS in enumerate(MASS_V):

    N_HNL = int(N_nu * BR_HNL_V[i] )

    HNL_KE_v = ENERGY_PION(MASS,N_HNL)
    #HNL_KE_v = ENERGY(N_HNL)
    # time distribution of HNL relative to neutrinos
    thnl_v = BUCKET(N_HNL) + RWM(SIGMA,N_HNL) + HNL_DELTATIME_v(MASS,HNL_KE_v,DISTANCE=400)
    # absolute time distribution of HNL
    #thnl_v = BUCKET(N_HNL) + RWM(SIGMA,N_HNL) + HNL_TIME_v(MASS,HNL_energy_v)
    
    HNL_KE_v_V.append(HNL_KE_v)
    thnl_v_V.append(thnl_v)

In [ ]:
TMIN = 0
TMAX = 3000
NBINS = 1*((TMAX-TMIN)+1)

BINS = np.linspace(TMIN,TMAX,NBINS)

NU_vals,NU_edges = np.histogram(tnu_v,bins=BINS)
NU_centers = 0.5*(NU_edges[1:]+NU_edges[:-1])
NU_errs = np.sqrt(NU_vals.astype(float))

HNL_centers_V = []
HNL_vals_V = []
HNL_errs_V = []

for i,MASS in enumerate(MASS_V):
    HNL_vals,HNL_edges = np.histogram(thnl_v_V[i],bins=BINS)
    HNL_centers = 0.5*(HNL_edges[1:]+HNL_edges[:-1])
    HNL_errs = np.sqrt(HNL_vals.astype(float))
                                               
    HNL_centers_V.append(HNL_centers)
    HNL_vals_V.append(HNL_vals)
    HNL_errs_V.append(HNL_errs)

In [ ]:
TMIN = 100
TMAX = 200
NBINS = 1*((TMAX-TMIN)+1)

BINS = np.linspace(TMIN,TMAX,NBINS)

NU_vals,NU_edges = np.histogram(tnu_v,bins=BINS)
NU_centers = 0.5*(NU_edges[1:]+NU_edges[:-1])
NU_errs = np.sqrt(NU_vals.astype(float))

fig = plt.figure(figsize=(12,6))
plt.errorbar(NU_centers,NU_vals,yerr=NU_errs,fmt='ko',label=r'$\nu$')
#plt.errorbar(HNL_centers,HNL_vals,yerr=HNL_errs,fmt='ro',label=r'HNL w/ Mass of %i MeV'%MASS)
plt.legend()
plt.yscale('log')
plt.xlabel('time [ns]')
plt.show()

In [ ]:
# energy spectrum of HNLs
fig = plt.figure(figsize=(12,6))
for i,MASS in enumerate(MASS_V):
    plt.hist(HNL_KE_v_V[i],bins=np.linspace(0,3000,100),histtype='step',lw=2,label='%i MeV'%MASS)
plt.xlabel('Enegy [MeV]')
plt.legend(loc=1)
plt.show()

In [ ]:
fig = plt.figure(figsize=(12,6))
for i,MASS in enumerate(MASS_V):
    plt.errorbar(HNL_centers_V[i],HNL_vals_V[i],yerr=HNL_errs_V[i],fmt='o',label=r'HNL w/ Mass of %i MeV'%MASS)
plt.legend()
plt.xlabel('time [ns]')
plt.show()

In [ ]:
# calculate fraction above 1.6 us
N1 = len(np.where(thnl_v > 1600)[0])
Ntot = float(len(thnl_v))
print ('frac > 1.6 us = %.03f'%((N1)/Ntot))

In [ ]:
import matplotlib.gridspec as gridspec

In [ ]:
fig = plt.figure(figsize=(18,6))
gs = gridspec.GridSpec(1, 3)

T1 = plt.subplot(gs[0])

T2 = plt.subplot(gs[1])

T3 = plt.subplot(gs[2])

TMIN = 0
TMAX = 100
NBINS = 10*((TMAX-TMIN)+1)

BINS = np.linspace(TMIN,TMAX,NBINS)

NU_vals,NU_edges = np.histogram(tnu_v,bins=BINS)
NU_centers = 0.5*(NU_edges[1:]+NU_edges[:-1])
NU_errs = np.sqrt(NU_vals.astype(float))
T1.errorbar(NU_centers,NU_vals,yerr=NU_errs,fmt='ko',label=r'$\nu$',markersize=3)

for i,MASS in enumerate(MASS_V):
    HNL_vals,HNL_edges = np.histogram(thnl_v_V[i],bins=BINS)
    HNL_centers = 0.5*(HNL_edges[1:]+HNL_edges[:-1])
    HNL_errs = np.sqrt(HNL_vals.astype(float))
    T1.errorbar(HNL_centers,HNL_vals,yerr=HNL_errs,fmt='o',label=r'HNL %i MeV'%MASS,markersize=3)

TMIN = 200
TMAX = 300
NBINS = 10*((TMAX-TMIN)+1)

BINS = np.linspace(TMIN,TMAX,NBINS)

NU_vals,NU_edges = np.histogram(tnu_v,bins=BINS)
NU_centers = 0.5*(NU_edges[1:]+NU_edges[:-1])
NU_errs = np.sqrt(NU_vals.astype(float))
T2.errorbar(NU_centers,NU_vals,yerr=NU_errs,fmt='ko',label=r'$\nu$',markersize=3)

for i,MASS in enumerate(MASS_V):
    HNL_vals,HNL_edges = np.histogram(thnl_v_V[i],bins=BINS)
    HNL_centers = 0.5*(HNL_edges[1:]+HNL_edges[:-1])
    HNL_errs = np.sqrt(HNL_vals.astype(float))
    T2.errorbar(HNL_centers,HNL_vals,yerr=HNL_errs,fmt='o',label=r'HNL %i MeV'%MASS,markersize=3)

#fig = plt.figure(figsize=(6,6))

TMIN = 1500
TMAX = 1600
NBINS = 10*((TMAX-TMIN)+1)

BINS = np.linspace(TMIN,TMAX,NBINS)

NU_vals,NU_edges = np.histogram(tnu_v,bins=BINS)
NU_centers = 0.5*(NU_edges[1:]+NU_edges[:-1])
NU_errs = np.sqrt(NU_vals.astype(float))
T3.errorbar(NU_centers,NU_vals,yerr=NU_errs,fmt='ko',label=r'$\nu$',markersize=3)

for i,MASS in enumerate(MASS_V):
    HNL_vals,HNL_edges = np.histogram(thnl_v_V[i],bins=BINS)
    HNL_centers = 0.5*(HNL_edges[1:]+HNL_edges[:-1])
    HNL_errs = np.sqrt(HNL_vals.astype(float))
    T3.errorbar(HNL_centers,HNL_vals,yerr=HNL_errs,fmt='o',label=r'%i MeV HNL'%MASS,markersize=3)

#fig = plt.figure(figsize=(6,6))

T2.legend(framealpha=1,loc=1,fontsize=14)
T2.set_xlabel('arrival time [ns]')
#plt.xlabel('time [ns]')
plt.show()

In [ ]:
fig = plt.figure(figsize=(12,6))

TMIN = 200
TMAX = 295
NBINS = 10*((TMAX-TMIN)+1)

BINS = np.linspace(TMIN,TMAX,NBINS)

NU_vals,NU_edges = np.histogram(tnu_v,bins=BINS)
NU_centers = 0.5*(NU_edges[1:]+NU_edges[:-1])
NU_errs = np.sqrt(NU_vals.astype(float))
plt.errorbar(NU_centers,NU_vals,yerr=NU_errs,fmt='bo',label=r'$\nu$ from BNB',markersize=8)

COLORS = ['r','orange']

for i,MASS in enumerate(MASS_V):
    HNL_vals,HNL_edges = np.histogram(thnl_v_V[i],bins=BINS)
    HNL_centers = 0.5*(HNL_edges[1:]+HNL_edges[:-1])
    HNL_errs = np.sqrt(HNL_vals.astype(float))
    plt.errorbar(HNL_centers,HNL_vals,yerr=HNL_errs,fmt='o',color=COLORS[i],\
                 label=r'HNL %i MeV'%MASS,markersize=8)


plt.gca().get_yaxis().set_ticks([])
plt.legend(framealpha=1,loc=1,fontsize=28)
plt.xlabel('arrival time [ns]',fontsize=28)
plt.ylim([0,2750])
plt.xlim([200,258])
#plt.title('HNL arrival time at MicroBooNE from BNB')
plt.tight_layout()
plt.savefig('HNLarrivalexample.pdf',dpi=250)
plt.show()